In [1]:
import tensorflow as tf
# tensorflow/examples/tutorials/mnist/input_data.py
from tensorflow.examples.tutorials.mnist import input_data
LOGDIR ='./mnist_demo/'
mnist = input_data.read_data_sets(LOGDIR + 'data', one_hot=True)

/Users/trungtv/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist_demo/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist_demo/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist_demo/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist_demo/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
def conv_layer(input, size_in, size_out):
    #kernel_shape = tf.Variable(tf.zeros([5,5, size_in, size_out]))
    kernel_shape = tf.Variable(tf.truncated_normal([5,5, size_in, size_out], stddev=0.1))
    bias = tf.Variable(tf.constant(0.1, shape=[size_out]))
    conv = tf.nn.conv2d(input, kernel_shape, strides=[1,1,1,1], padding='SAME')
    act = tf.nn.relu(conv + bias)
    maxpool = tf.nn.max_pool(act, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    return maxpool

In [5]:
def fc_layer(input, size_in, size_out):
    #w = tf.Variable(tf.constant(0.0, shape=[size_in, size_out]))
    w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
    b = tf.Variable(tf.constant(0.1, shape=[size_out]))
    act = tf.matmul(input, w) + b # NOT tf.nn.sigmoid?
    return act

In [6]:
# MAKING THE GRAPH
sess = tf.Session()
x = tf.placeholder(tf.float32, shape=[None, 28 * 28 * 1]) #because we used read_data_sets
y = tf.placeholder(tf.float32, shape=[None, 10])
x_image = tf.reshape(x, [-1, 28, 28, 1])

conv1 = conv_layer(x_image, 1, 32)
conv2 = conv_layer(conv1, 32, 64)
conv2_reshape = tf.reshape(conv2, [-1, 7 * 7 * 64])
fc1 = fc_layer(conv2_reshape, 7 * 7 * 64, 1024)
fc1_relu = tf.nn.relu(fc1)

fc2 = fc_layer(fc1_relu, 1024, 10)
# compute the cross entropy (xent)
xent = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=fc2, labels=y))
correct_prediction = tf.equal(tf.argmax(fc2,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# define our optimizer
optimizer = tf.train.AdamOptimizer(1e-4).minimize(xent)

RuntimeError: tf.placeholder() is not compatible with eager execution.

In [ ]:
sess.run(tf.global_variables_initializer())
# train for 1000 steps
for i in range(1000):
    batch = mnist.train.next_batch(100)
    _, xent_value = sess.run([optimizer, xent], feed_dict={x: batch[0], y: batch[1]})
    if i % 50 == 0:
        print("Step {} Xent = {}".format(i,xent_value))
        [train_accuracy] = sess.run([accuracy], feed_dict={x: batch[0], y: batch[1]})
        print("Step {} Accuracy = {}".format(i, train_accuracy))